<a href="https://colab.research.google.com/github/manu675/Data_Mining_Cup_Prudsys_21/blob/master/data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining Cup Prudsys 2021


Task: https://www.data-mining-cup.com/dmc-2021/?utm_source=dmc&utm_medium=email&utm_campaign=newsletter

In [1]:
# importing libaries:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.naive_bayes import GaussianNB

from collections import Counter

from scipy.stats import loguniform, randint, uniform

#import imblearn
#print(imblearn.__version__)

#from imblearn.over_sampling import RandomOverSampler
from sklearn.utils import class_weight



## Reading in the data

In [ ]:
## Way 2: uploading the local data to google colab
# importing local data to google colab:
from google.colab import files
uploaded = files.upload()

In [ ]:
## Way 2: uploading the local data to google colab
# importing local data to google colab:
from google.colab import files
uploaded = files.upload()

In [ ]:
## Way 2: uploading the local data to google colab
# importing local data to google colab:
from google.colab import files
uploaded = files.upload()

In [2]:
eval_df = pd.read_csv("data/evaluation.csv", sep =";")

In [3]:
items_df = pd.read_csv("data/items.csv", sep ="|")

In [4]:
transactions_df = pd.read_csv("data/transactions.csv", sep ="|")

In [5]:
eval_df.head()

,itemID
0,12
1,45274
2,10104
3,41371
4,14015


In [6]:
eval_df.shape

(1000, 1)

In [7]:
items_df.head()

,itemID,title,author,publisher,main topic,subtopics
0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH]
1,73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,"[5AJ,AGZ,WFA,YBG,YBL,YNA,YPA]"
2,19194,Red Queen 1,Victoria Aveyard,Orion Publishing Group,YFH,"[5AP,FBA]"
3,40250,Meine Kindergarten-Freunde (Pirat),NaN,Ars Edition GmbH,YB,"[5AC,5AD,YBG,YBL,YF]"
4,46107,Mein großes Schablonen-Buch - Wilde Tiere,Elizabeth Golding,Edition Michael Fischer,WFTM,"[WD,WFTM,YBG,YBL,YBLD,YBLN1]"


In [8]:
items_df.shape

(78030, 6)

In [9]:
transactions_df.head()

,sessionID,itemID,click,basket,order
0,0,21310,1,0,0
1,1,73018,1,0,0
2,2,19194,1,0,0
3,3,40250,1,0,0
4,4,46107,1,0,0


In [10]:
transactions_df.shape

(365143, 5)

In [11]:
#Count of each items occurence in transactions
transactions_df['itemID'].value_counts().sort_values(ascending=False).to_frame()

,itemID
58522,2291
66247,1775
10666,1749
44220,1687
69185,1290
...,...
54297,1
10366,1
14587,1
4350,1


In [12]:
#Count of each unique value in eval_df
eval_df['itemID'].value_counts().sort_values(ascending=False).to_frame()

,itemID
78680,1
48231,1
2054,1
18441,1
45116,1
...,...
30713,1
47098,1
71619,1
57282,1


In [13]:
#Left join transactions_df on eval_df to see how often each evaluation item occurs in transactions
#transactions_df.join(eval_df, on='itemID', how='right', lsuffix = '_trans', rsuffix = '_eval')
eval_trans_df = pd.merge(eval_df,transactions_df, on='itemID',how='left')
eval_trans_df

,itemID,sessionID,click,basket,order
0,12,NaN,NaN,NaN,NaN
1,45274,NaN,NaN,NaN,NaN
2,10104,NaN,NaN,NaN,NaN
3,41371,138327.0,1.0,0.0,0.0
4,41371,214045.0,1.0,1.0,0.0
...,...,...,...,...,...
8202,68157,270888.0,1.0,0.0,0.0
8203,44964,36017.0,1.0,0.0,0.0
8204,12971,124023.0,1.0,0.0,0.0
8205,12971,226727.0,1.0,0.0,0.0


In [14]:
#Count occurence of each evaluation item in transactions_df
#Key: ocurence = total occurence of item in session, 
#     order = total ordered units of item, 
#     basket = sum of occurences of this item in shopping basket
eval_trans_df.loc[:,'occurence'] = 1
eval_trans_df.loc[eval_trans_df['sessionID'].isnull(),'occurence'] = 0

occurence_df = eval_trans_df[['itemID','occurence','order','basket']].groupby(['itemID']).agg('sum')
occurence_df

,occurence,order,basket
itemID,,,
12,0,0.0,0.0
259,0,0.0,0.0
337,0,0.0,0.0
457,0,0.0,0.0
468,20,3.0,1.0
...,...,...,...
78486,0,0.0,0.0
78675,7,0.0,0.0
78680,0,0.0,0.0


In [15]:
#Group items by their occurence in any session: 496 out of 1000 eval items are not features in transactions_df!
occurence_df[['occurence','order']].groupby(['occurence']).count().rename(columns={'order':'items_count'})

,items_count
occurence,
0,496
1,153
2,70
3,45
4,38
5,19
6,20
7,19
8,11
